# Exercise 4
# Text Processing to filter spam messages

## <b>Problem Description</b>
To filter spam messages using text processing

### Step 1

Import the required packages<br>
We will be using pandas to read the data frame, nltk for lemmatizing/stemming the text and sklearn for splitting it into testing and training data

In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

c:\users\balaj\appdata\local\programs\python\python36\python.exe
3.6.2 (v3.6.2:5fd33b5, Jul  8 2017, 04:57:36) [MSC v.1900 64 bit (AMD64)]
sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)


In [2]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn import svm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\balaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Step 2

Read the spam CSV file into a panda data frame

In [3]:
data = pd.read_csv("CSV/spam.csv", encoding = "latin-1")
data = data[['v1', 'v2']]
data = data.rename(columns = {'v1': 'label', 'v2': 'text'})
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Step 3

List the number of ham and spam values

In [4]:
data.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

### Step 4

Define two functions review_messages and alternate_review_messages which will help in removing the stopwords

In [5]:
def review_messages(msg):
    # converting messages to lowercase
    msg = msg.lower()
    return msg

lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))
def alternative_review_messages(msg):
    # converting messages to lowercase
    msg = msg.lower()
    # removing stopwords 
    msg = [word for word in msg.split() if word not in stopwords]
    # using a lemmatized 
    msg = " ".join([lemmatizer.lemmatize(word) for word in msg])
    return msg

### Step 5

Normalize the given data and split it into testing and training data


In [6]:
data['text'] = data['text'].apply(review_messages)
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.1, random_state = 1)
# training the vectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

### Step 6

Fit the same using a SVM model

In [7]:
svm = svm.SVC(C=1000)
svm.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### Step 8

Print the confusion matrix

In [8]:
from sklearn.metrics import confusion_matrix
X_test = vectorizer.transform(X_test)
y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))

[[490   0]
 [ 10  58]]


### Step 9

Define a function called as predict to predict whether it is spam or ham

In [9]:
def predict(msg):
    msg = vectorizer.transform([msg])
    prediction = svm.predict(msg)
    return prediction[0]


### Step 10

Test the model

In [10]:
predict("My name is balaji")

'ham'

In [11]:
predict("You HAVE won 1545 billion as CASH PRIZE!!")

'spam'